# Notebook with Query and WH Performance Analysis Queries

- Adjust the date range before execution.

- Adjust the or names patterns for the Warehouses, Tasks, and Schemas as needed.

In [ ]:
SET QUERY_START_DATE = '2025-05-25' ;
SET QUERY_END_DATE = '2025-05-26' ;
SET WAREHOUSES_PATTERN = 'SNOWADHOC'; -- wildcard supported
SET TASKS_PATTERN = '%'; -- wildcard supported
SET SCHEMAS_PATTERN = '%'; -- wildcard supported

# Concurrency

## Description:

Get the number of concurrent jobs on a warehouse cluster.

## How to Interpret Results:

Returns the number of active jobs per cluster per minute.

In [ ]:
WITH job
--Peak concurrent job count by minute and cluster number
AS (
	SELECT
		job.START_TIME
		, TO_NUMBER(EXECUTION_TIME, 18, 3) AS dur_xp_executing_ms
		, dateadd(ms, -1*execution_time::number(38,10), end_time) as xp_start_time
		, END_TIME AS xp_end_time
		, cluster_number AS cluster_no
		, job.QUERY_ID
	FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY job
	WHERE WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN
    AND
		START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE
        and cluster_number > 0
	)
, time_in_ms AS (
	--    select dateadd(ms, seq8(), dateadd(hour, -24, current_timestamp()))    as time_ms
	--    from table(generator(rowcount => 24 * 60 * 60 * 1000)) t --  2 hours
	SELECT xp_start_time AS time_ms, cluster_no
	FROM job
	UNION
	SELECT xp_end_time AS time_ms, cluster_no
	FROM job
)
	SELECT date_trunc('minute', time_ms) AS minute
		, cluster_no
		--, any_value(hash(sec::string, warehouse_name, cluster_no::string)) as key
		, max(job_active_count) AS max_job_active_count_ms
	FROM (
		SELECT time_ms
			, t.cluster_no
			, count(QUERY_ID) AS job_active_count
		FROM time_in_ms t -- roll up to ms level
			--left join job
		INNER JOIN job ON time_ms >= xp_start_time
			AND time_ms < xp_end_time
            and job.cluster_no = t.cluster_no
		GROUP BY 1, 2
		)
	GROUP BY 1, 2
;


# Scaling

## Description:

Two separate queries that list out the warehouses and times that could benefit from either a MCW setting OR scaling up to a larger size

## How to Interpret Results:

Use this list to determine reconfiguration of a warehouse and the times or users that are causing contention on the warehouse


### Scale out

In [ ]:
SELECT TO_DATE(START_TIME) as DATE

,WAREHOUSE_NAME

,SUM(AVG_RUNNING) AS SUM_RUNNING

,SUM(AVG_QUEUED_LOAD) AS SUM_QUEUED

FROM "SNOWFLAKE"."ACCOUNT_USAGE"."WAREHOUSE_LOAD_HISTORY"

WHERE 

START_TIME BETWEEN $QUERY_START_DATE :: DATE
                AND $QUERY_END_DATE :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


GROUP BY 1,2

HAVING SUM(AVG_QUEUED_LOAD) >0

;

### Scale up

In [ ]:
SELECT QUERY_ID

,USER_NAME

,WAREHOUSE_NAME

,WAREHOUSE_SIZE

,BYTES_SCANNED

,BYTES_SPILLED_TO_REMOTE_STORAGE

,BYTES_SPILLED_TO_REMOTE_STORAGE / BYTES_SCANNED AS SPILLING_READ_RATIO

FROM "SNOWFLAKE"."ACCOUNT_USAGE"."QUERY_HISTORY"

WHERE BYTES_SPILLED_TO_REMOTE_STORAGE > BYTES_SCANNED * 5 -- Each byte read was spilled 5x on average

AND

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


ORDER BY SPILLING_READ_RATIO DESC

;

# Warehouse Cache Usage

## Description:
Aggregate across all queries broken out by warehouses showing the percentage of data scanned from the warehouse cache.
## How to Interpret Results:
Look for warehouses that are used from querying/reporting and have a low percentage. This indicates that the warehouse is suspending too quickly



In [ ]:

SELECT WAREHOUSE_NAME
,COUNT(*) AS QUERY_COUNT
,SUM(BYTES_SCANNED) AS BYTES_SCANNED
,SUM(BYTES_SCANNED*PERCENTAGE_SCANNED_FROM_CACHE) AS BYTES_SCANNED_FROM_CACHE
,SUM(BYTES_SCANNED*PERCENTAGE_SCANNED_FROM_CACHE) / SUM(BYTES_SCANNED) AS PERCENT_SCANNED_FROM_CACHE
FROM "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY
WHERE 
 BYTES_SCANNED > 0
 
AND 

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME   LIKE $WAREHOUSES_PATTERN


GROUP BY 1
ORDER BY 5


# Heavy scanners
## Description:
Ordered list of queries that run queries that scan a lot of data.
## How to Interpret Results:
This is a potential opportunity to identify queries which read a lot of data and if filtering is selective enough or caching is working properly.



In [ ]:
select 
  query_id
, warehouse_name
, avg(case when partitions_total > 0 then partitions_scanned / partitions_total else 0 end) avg_pct_scanned
from   "SNOWFLAKE"."ACCOUNT_USAGE".query_history
where  

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


group by 1, 2
order by 3 desc
;


# Spillage

## Description:

Identifies the top 10 worst offending queries in terms of bytes spilled to remote storage.

## How to Interpret Results:

These queries should most likely be run on larger warehouses that have more local storage and memory.


In [ ]:
select query_id, substr(query_text, 1, 50) partial_query_text, user_name, warehouse_name, warehouse_size,

BYTES_SPILLED_TO_REMOTE_STORAGE, start_time, end_time, total_elapsed_time/1000 total_elapsed_time

from "SNOWFLAKE"."ACCOUNT_USAGE".query_history

where BYTES_SPILLED_TO_REMOTE_STORAGE > 0

AND 

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


order by BYTES_SPILLED_TO_REMOTE_STORAGE desc

limit 10

;

# Query Volume

## Description:

Shows average number of queries run on an hourly basis to help better understand typical query activity.

## How to Interpret Results:

How many queries are being run on an hourly basis? Is this more or less than we anticipated? What could be causing this?


In [ ]:
SELECT DATE_TRUNC('HOUR', START_TIME) AS QUERY_START_HOUR

,WAREHOUSE_NAME

,COUNT(*) AS NUM_QUERIES

FROM "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY

WHERE 

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


GROUP BY 1, 2

ORDER BY 1 DESC, 2

;

# Longest Running Queries

## Description:

Looks at the top 50 longest running queries to see if there are patterns

## How to Interpret Results:

Is there an opportunity to optimize with clustering or upsize the warehouse?


In [ ]:
select


QUERY_ID

,QUERY_TEXT

,TOTAL_ELAPSED_TIME/1000 AS QUERY_EXECUTION_TIME_SECONDS

,compilation_time/1000 AS COMPILATION_TIME_SECONDS

,execution_time/1000 AS EXECUTION_TIME_SECONDS

,queued_overload_time/1000 AS QUEUED_TIME_SECONDS

from "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY Q

where 1=1

AND

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN



and TOTAL_ELAPSED_TIME > 0 --only get queries that actually used compute

and ERROR_CODE iS NULL


order by TOTAL_ELAPSED_TIME desc


LIMIT 50

;

# Longest Compilation Times

## Description:

Looks at the top 50 longest compiling queries

## How to Interpret Results:

Is there an opportunity to reduce query complexity?

In [ ]:
select


QUERY_ID

,QUERY_TEXT

,TOTAL_ELAPSED_TIME/1000 AS QUERY_EXECUTION_TIME_SECONDS

,COMPILATION_TIME/1000 AS QUERY_COMPILATION_TIME_SECONDS

,QUERY_COMPILATION_TIME_SECONDS/QUERY_EXECUTION_TIME_SECONDS AS PERCENT_COMPILATION_TIME_SECONDS

from "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY Q

where 1=1

AND

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN

and COMPILATION_TIME > 2000 -- greater than 2 seconds

and TOTAL_ELAPSED_TIME > 3000 -- only look at long running queries

and ERROR_CODE iS NULL


order by PERCENT_COMPILATION_TIME_SECONDS desc


LIMIT 50

;

# Longest Query Text
## Description
List the Top 50 queries witht he longest text.
## How to Interpret Results
Find candidates for refactoring.

In [ ]:
select


QUERY_ID

,QUERY_TEXT

,LENGTH(QUERY_TEXT) AS QUERY_LENGTH


from "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY Q

where 1=1

AND

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN


and QUERY_LENGTH >= 100000 --only get queries longer than n characters (modify as needed)

and ERROR_CODE iS NULL

and QUERY_TEXT is not null


order by QUERY_LENGTH desc


LIMIT 50

;

# Queries Scanning the Most Data

## Description:

Looks at the top 50 queries that scan the largest number of micro partitions

## How to Interpret Results:

Is there an opportunity to optimize with clustering or upsize the warehouse?


In [ ]:
select


QUERY_ID

,ROW_NUMBER() OVER(ORDER BY PARTITIONS_SCANNED DESC) as QUERY_ID_INT

,QUERY_TEXT

,TOTAL_ELAPSED_TIME/1000 AS QUERY_EXECUTION_TIME_SECONDS

,PARTITIONS_SCANNED

,PARTITIONS_TOTAL


from "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY Q

where 1=1

and

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN



and TOTAL_ELAPSED_TIME > 0 --only get queries that actually used compute

and ERROR_CODE iS NULL

and PARTITIONS_SCANNED is not null


order by PARTITIONS_SCANNED desc


LIMIT 50

;

# Query Duration Performance Goals

## Description:

Find the distribution of total elapsed times for all queries in a warehouse for a period.

## How to Interpret Results:

Do we meet the performance objective per query? For example, if we expect queries to complete in less than 5 seconds, then this usually means that the p95 or p99 values are below 5 seconds.


In [ ]:
select

min(total_elapsed_time) min,

max(total_elapsed_time) max,

median(total_elapsed_time) median,

avg(total_elapsed_time) avg,

percentile_cont(0.95) within group (order by total_elapsed_time) p95,

percentile_cont(0.80) within group (order by total_elapsed_time) p80,

percentile_cont(0.65) within group (order by total_elapsed_time) p65

from "SNOWFLAKE"."ACCOUNT_USAGE".query_history

WHERE

START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN
AND query_type = 'SELECT'

AND execution_status = 'SUCCESS'

AND not contains(query_text, 'snowflake')

AND not contains(query_text, 'SNOWFLAKE')

order by 1;

# Queueing

## Description:

Query load value for queries queued because the warehouse was overloaded. 

## How to Interpret Results:

Which warehouses have a high query load?


In [ ]:
SELECT TO_TIME(START_TIME) as DATE
,WAREHOUSE_NAME
,SUM(AVG_RUNNING) AS SUM_RUNNING
,SUM(AVG_QUEUED_LOAD) AS SUM_QUEUED
,SUM_QUEUED /  SUM_RUNNING AS QUEUE_LOAD
FROM "SNOWFLAKE"."ACCOUNT_USAGE"."WAREHOUSE_LOAD_HISTORY"
WHERE 


START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN

GROUP BY 1,2
HAVING SUM(AVG_QUEUED_LOAD) >0
ORDER BY 1
;

# Credit Consumption

## Description:

Get how many credits have been spent by differents services (Warehouses, pipes, etc)

## How to Interpret Results:

Are there any outliers in credit consumption that need to be addressed?


In [ ]:
  SELECT
                DATE_TRUNC('HOUR', START_TIME) AS START_TIME,
                NAME,
                SERVICE_TYPE,
                SUM(CREDITS_USED) AS CREDITS_USED,
                SUM(CREDITS_USED_COMPUTE) AS CREDITS_COMPUTE,
                SUM(CREDITS_USED_CLOUD_SERVICES) AS CREDITS_CLOUD
            FROM
                "SNOWFLAKE"."ACCOUNT_USAGE".METERING_HISTORY
            WHERE
                START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                AND $QUERY_END_DATE  :: DATE

                AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN

            GROUP BY
                START_TIME,
                NAME,
                SERVICE_TYPE;

# Credits Consumption by User and Warehouse

## Description:

Get approximately how many credits have spent users in a specific timeframe on the specified warehouses.

## How to Interpret Results:

Which users use the most credits on which WH?



In [ ]:
  WITH USER_HOUR_EXECUTION_CTE AS (
                SELECT
                        USER_NAME,
                        WAREHOUSE_NAME,
                        DATE_TRUNC('HOUR', START_TIME) AS START_TIME_HOUR,
                        SUM(EXECUTION_TIME) AS USER_HOUR_EXECUTION_TIME
                FROM
                        "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY
                WHERE
                        WAREHOUSE_NAME IS NOT NULL                   
                        AND EXECUTION_TIME > 0
                        AND
                        START_TIME BETWEEN $QUERY_START_DATE  :: DATE
                        AND $QUERY_END_DATE  :: DATE
                        AND WAREHOUSE_NAME  LIKE $WAREHOUSES_PATTERN         
                GROUP BY
                        USER_NAME,
                        WAREHOUSE_NAME,
                        START_TIME_HOUR
            ),
            HOUR_EXECUTION_CTE AS (
                SELECT
                        START_TIME_HOUR,
                        WAREHOUSE_NAME,
                        SUM(USER_HOUR_EXECUTION_TIME) AS HOUR_EXECUTION_TIME
                FROM
                        USER_HOUR_EXECUTION_CTE
                GROUP BY
                        START_TIME_HOUR,
                        WAREHOUSE_NAME
            ),
            APPROXIMATE_CREDITS AS (
                SELECT
                        UHE.USER_NAME,
                        WMH.WAREHOUSE_NAME,
                        (
                                UHE.USER_HOUR_EXECUTION_TIME / HE.HOUR_EXECUTION_TIME
                        ) * WMH.CREDITS_USED AS APPROXIMATE_CREDITS_USED
                FROM
                        USER_HOUR_EXECUTION_CTE UHE
                        JOIN HOUR_EXECUTION_CTE HE ON UHE.START_TIME_HOUR = HE.START_TIME_HOUR
                        AND HE.WAREHOUSE_NAME = UHE.WAREHOUSE_NAME
                        JOIN "SNOWFLAKE"."ACCOUNT_USAGE".WAREHOUSE_METERING_HISTORY WMH ON WMH.WAREHOUSE_NAME = UHE.WAREHOUSE_NAME
                        AND WMH.START_TIME = UHE.START_TIME_HOUR
            )
            SELECT
                USER_NAME,
                WAREHOUSE_NAME,
                SUM(APPROXIMATE_CREDITS_USED) AS APPROXIMATE_CREDITS_USED
            FROM
                APPROXIMATE_CREDITS
            GROUP BY
                USER_NAME,
                WAREHOUSE_NAME
            ORDER BY
                APPROXIMATE_CREDITS_USED DESC
           ;

# Warehouse Roles

## Description:

List which roles have access to a WH

## How to Interpret Results:

Get a list of roles for a WH and look up on the PowerBI dashboard which users have these roles: 
[Snowflake PowerBI Dashboard](https://app.powerbi.com/groups/me/apps/4d286c72-a618-4f78-8f0a-827ace465ed5/reports/6ab19790-952e-4448-8a9e-c578754efad3?experience=power-bi)

In [ ]:
show grants on warehouse $WAREHOUSES_PATTERN;

# Repeated Queries

## Description:

Get how many times does a query have been executed and the max duration of execution time.

## How to Interpret Results:

Analyse variance of repeated queries.
  


In [ ]:

SELECT
    QUERY_TEXT,
    COUNT(*) AS NUMBER_OF_QUERIES,
    min(total_elapsed_time/1000) min, 
    max(total_elapsed_time/1000) max, 
    median(total_elapsed_time/1000) median, 
    avg(total_elapsed_time/1000) avg, 
    percentile_cont(0.95) within group (order by total_elapsed_time/1000) p95,
    percentile_cont(0.80) within group (order by total_elapsed_time/1000) p80,
    percentile_cont(0.65) within group (order by total_elapsed_time/1000) p65

FROM
    "SNOWFLAKE"."ACCOUNT_USAGE".QUERY_HISTORY Q
WHERE
    TOTAL_ELAPSED_TIME > 0
    AND
    START_TIME BETWEEN $QUERY_START_DATE  :: DATE
    AND $QUERY_END_DATE  :: DATE

    AND WAREHOUSE_NAME LIKE $WAREHOUSES_PATTERN
    
GROUP BY
    QUERY_TEXT
HAVING
    COUNT(*) >= 5 -- set minimum number of executions  
ORDER BY
    NUMBER_OF_QUERIES DESC
;

# Failed Tasks

## Description:
Returns a list of task executions that failed.

## How to Interpret Results:
Revisit these task executions to resolve the errors.

In [ ]:
select *
  from "SNOWFLAKE"."ACCOUNT_USAGE".task_history
  WHERE STATE = 'FAILED'

AND
 START_TIME BETWEEN $QUERY_START_DATE  :: DATE
    AND $QUERY_END_DATE  :: DATE

    AND 
    NAME LIKE $TASKS_PATTERN -- Task name
    AND SCHEMA_NAME LIKE $SCHEMAS_PATTERN -- Schema name

  
  order by query_start_time DESC
  ;

# Long Running Tasks

## Description:
Returns an ordered list of the longest running tasks

## How to Interpret Results:
revisit task execution frequency or the task code for optimization

In [ ]:
select DATEDIFF(seconds, QUERY_START_TIME,COMPLETED_TIME) as DURATION_SECONDS
                ,*
from "SNOWFLAKE"."ACCOUNT_USAGE".task_history
WHERE STATE = 'SUCCEEDED'

AND
 START_TIME BETWEEN $QUERY_START_DATE  :: DATE
    AND $QUERY_END_DATE  :: DATE

    AND 
    NAME LIKE $TASKS_PATTERN -- Task name
    AND SCHEMA_NAME LIKE $SCHEMAS_PATTERN -- Schema name

    
order by DURATION_SECONDS desc
  ;